In [1]:
# Standard library imports
import json
import os
import random
import sys
from pprint import pprint

# Third party imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import Bbox, inset_axes
from scipy.special import expit
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_curve,
)
from sklearn.model_selection import StratifiedShuffleSplit
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_sparse import coalesce
from tqdm import tqdm

# Local application imports
sys.path.append("/pbs/home/e/erodrigu/TesisPhDEzequielRodriguez/Code")
from my_utils.gnn_dataset_utilities import (
    MaskNode,
    PruneGraph2FirstCrown,
    SD433UMDatasetPyG,
    get_station_features,
    plot_graph
)
from my_utils.my_basic_utils import (
    add_zenith_related_columns,
    create_bins,
    filter_dataframe,
    print_banner_text,
)
from my_utils.my_style import MyStyle
from my_utils.torch_models_and_utilities import (
    FocalLoss,
    GNNDiscriminator,
    ROC_resam,
    TraceAnalyzer,
    evaluate_model_to_dataframe,
    get_accuracy,
)

# set PATHS
code_PATH = os.path.abspath(os.path.join(".."))
project_PATH = os.path.abspath(os.path.join(code_PATH, ".."))
data_PATH = os.path.join(project_PATH, "data")

ModuleNotFoundError: No module named 'torch'

### Version check

In [ ]:
print(f"Torch version: {torch.__version__}")
print(f"Torch CUDA version: {torch.version.cuda}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is: {device}")

In [ ]:
!nvidia-smi

### Dataset Index Loading

In [ ]:
folder_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/"
dir_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/JSONfiles/"
index = pd.DataFrame()

# indexes
primaries = ["Proton", "Photon"]
energy_bins = ["16.5_17.0", "17.0_17.5"]
atms = ["01", "03", "08", "09"]
indexes = [
    f"index_hadron_rec_{x}_{y}_{z}.csv"
    for x in primaries
    for y in energy_bins
    for z in atms
]

# create the index by appending
for index_name in indexes:
    proton_index = pd.read_csv(folder_path + index_name, on_bad_lines="skip")
    photon_rec_index = pd.read_csv(
        folder_path + index_name.replace("hadron", "photon"), on_bad_lines="skip"
    )
    index_ = pd.merge(
        proton_index,
        photon_rec_index,
        on=["filename", "atm_model", "shower_id", "use_id"],
        how="left",
    )
    index = pd.concat([index, index_], ignore_index=True)

index = index.drop_duplicates()
index = index.drop_duplicates(subset=["filename"])
# we won't train using iron
index["mass_group"] = index["filename"].str.split(pat="_", expand=True)[0]
index = index[index["mass_group"] != "Iron"]
print(f"Events before quality cuts: {len(index)}")

In [ ]:
index.describe()

### Quality Cuts and Binning

In [ ]:
index = index.sample(frac=1)
index.loc[index["filename"].str.contains("Photon"), "isPhoton"] = 1
index.loc[index["filename"].str.contains("Proton"), "isPhoton"] = 0

index["sin2zenith"] = np.sin(index["zenithMC"]) ** 2
# photon efficiency from fit from simulations
index["est_efficiency"] = (
    15.4074
    + 17.4996 * (np.log10(index["energyMC"]) - 17)
    - 12.7485 * index["sin2zenith"]
    - 20.7650 * index["sin2zenith"] ** 2
    - 13.1239 * (np.log10(index["energyMC"]) - 17) * index["sin2zenith"]
)
index["est_efficiency"] = expit(index["est_efficiency"])

feature_filters = {
    "zenithMC": {"filter_type": "range", "max_cut": np.deg2rad(45)},
    "photon_energy": {"filter_type": "range", "min_cut": 1},
#    "est_efficiency": {"filter_type": "range", "min_cut": 0.9},
    "isT5": {"filter_type": "value", "value_to_keep": 1}
    #"is5T5": {"filter_type": "value", "value_to_keep": 1},
    #"is6T5": {"filter_type": "value", "value_to_keep": 1},

}
index = filter_dataframe(index, feature_filters)

index, e_bin_centers, e_bin_edges, e_labels = create_bins(
    index,
    lower_val=10**16.5,
    upper_val=10**17.5,
    num=6,
    unbinned_col="energyMC",
    bin_column_name="e_bin",
    bin_width="equal_logarithmic",
)

index, z_bin_centers, z_bin_edges, z_labels = create_bins(
    index,
    lower_val=0,
    upper_val=np.sin(np.deg2rad(45)) ** 2,
    num=4,
    unbinned_col="sin2zenith",
    bin_column_name="z_bin",
    bin_width="equal",
)

index = index.loc[~index["e_bin"].isnull()]

# problems in ADSTs
exclude_list = [
"Photon_17.0_17.5_011102_11",
"Photon_17.0_17.5_080595_20"
]
index = index[~index['filename'].isin(exclude_list)]

print(f"Events after quality cuts: {len(index)}")

### Balanced Dataset Division

In [ ]:
# def get_fatt_ph(sec, dsec):
#     def func(x, p0, p1, p2, p3):
#         return p0 * (1 + (x - p2) / p1)**(p1 / p3) * np.exp(-(x - p2) / p3)

#     # Parameters and their errors
#     popt = [3.88940e+01, 6.23117e-01, 1.00999e+00, 9.04938e-02]
#     perr = [5.62431e-02, 5.26757e-02, 2.32738e-03, 4.79750e-03]

#     # Evaluate the function
#     f = func(sec, *popt)

#     # Unpack parameters and their errors
#     f0, f1, f2, f3 = popt
#     df0, df1, df2, df3 = perr

#     # Calculate the error
#     factor = 1 + (sec - f2) / f1
#     var0 = (f * df0 / f0)**2
#     var1 = (f * (np.log(factor) / f3 - (sec - f2) / (f1 * f3 * factor)) * df1)**2
#     var2 = (f * (f2 - sec) * df2 / (f3 * (f2 - sec - f1)))**2
#     var3 = (f * (f1 * np.log(factor) - sec + f2) * df3 / f3**2)**2
#     var4 = (-f * (sec - f2) * dsec / (f3 * (sec - f2 + f1)))**2
#     df = np.sqrt(var0 + var1 + var2 + var3 + var4)

#     return [f, df]

# def get_norm_fatt_ph(sec, dsec):
#     v_fatt_ph = get_fatt_ph(sec, dsec)
#     secref = np.cos(30 * np.pi / 180)**-1
#     norm = get_fatt_ph(secref, 0)[0]
#     v = [v_fatt_ph[0] / norm, v_fatt_ph[1] / norm]
#     return v

# def get_fatt_pr(x, dx):
#     def func(x, a, b, c):
#         return 1 + a * x + b * x**2 + c * x**3

#     # Parameters and their errors
#     popt = [1.88357, -1.74331, -3.45984]
#     perr = [2.81332e-03, 1.45271e-02, 5.38753e-02]

#     # Evaluate the function
#     f = func(x, *popt)

#     # Unpack parameters and their errors
#     f1, f2, f3 = popt
#     df1, df2, df3 = perr

#     # Calculate the error
#     var1 = (x * df1)**2
#     var2 = (x**2 * df2)**2
#     var3 = (x**3 * df3)**2
#     varx = ((f1 + 2 * f2 * x + 3 * f3 * x**2) * dx)**2
#     df = np.sqrt(var1 + var2 + var3 + varx)

#     return [f, df]

# def get_photon_norm():
#     secref = np.cos(30 * np.pi / 180)**-1
#     photon_norm = get_fatt_ph(secref, 0)[0]
#     return photon_norm

# photon_norm = get_photon_norm()

# x = np.cos(np.deg2rad(30))**2 - np.cos(np.deg2rad(30))**2
# dx = 1
# alpha = 1.15 * (1 - 0.192 * pow(pow(np.cos(np.deg2rad(30)), 2), 2.96))
# eq_energy = pow(10,16) * pow((38.9545 / (get_fatt_pr(x, dx)[0]*photon_norm)), (1/alpha))

In [ ]:
# Combine the two categorical variables for stratified sampling
index["categorical_balance"] = (
    index["isPhoton"].astype(str)
    + "_"
    + index["e_bin"].astype(str)
    + "_"
    + index["z_bin"].astype(str)
)

random_seed = 42
stratified_split = StratifiedShuffleSplit(
    n_splits=1, test_size=0.25, random_state=random_seed
)

for dev_index_, test_index_ in stratified_split.split(
    index, index["categorical_balance"]
):
    # Original Training set
    dev_index = index.iloc[dev_index_]

    # Testing set
    test_index = index.iloc[test_index_]

# Further split the original training set into train and validation sets
validation_size = 0.25  # Adjust as needed
split = StratifiedShuffleSplit(
    n_splits=1, test_size=validation_size, random_state=random_seed
)

for train_index_, validation_index_ in split.split(
    dev_index, dev_index["categorical_balance"]
):
    train_index = dev_index.iloc[train_index_]
    validation_index = dev_index.iloc[validation_index_]

# Print the size of each dataset
print("Train dataset size:", train_index.shape[0])
print("Validation dataset size:", validation_index.shape[0])
print("Test dataset size:", test_index.shape[0])

### Generation of Normalization Dictionary

In [ ]:
dir_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/JSONfiles/"
root_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/root/"

test_run = True
n_train_evts = 1000 if test_run else len(train_index)
n_val_evts = 0 if test_run else len(validation_index)
n_test_evts = 0 if test_run else len(test_index)

# set paths according to index
train_paths = list(train_index.iloc[:n_train_evts, 0].values)
train_paths = list(map(lambda item: dir_path + item + ".json", train_paths))
val_paths = list(validation_index.iloc[:n_val_evts, 0].values)
val_paths = list(map(lambda item: dir_path + item + ".json", val_paths))
test_paths = list(test_index.iloc[:n_test_evts, 0].values)
test_paths = list(map(lambda item: dir_path + item + ".json", test_paths))

In [ ]:
dict_already_computed = True
if dict_already_computed:
    normalization_dict = {
 'WCD_signal': {'mean': 0.9242377699753578,
                'method': 'standardization',
                'std': 0.7037094813650729},
 'deltaTimeHottest': {'mean': -47.27531568592806,
                      'method': 'standardization',
                      'std': 603.6062483849028},
 'effective_area': {'max': 31.380000000000003,
                    'method': 'min_max_scaling',
                    'min': 0},
 'pmt_number': {'max': 3, 'method': 'min_max_scaling', 'min': 1},
 'rho_mu': {'mean': 0.21510971141596952,
            'method': 'standardization',
            'std': 0.9057438827119321},
 'x': {'mean': -1.01887806305201,
       'method': 'standardization',
       'std': 364.5171135403138},
 'y': {'mean': -0.06542848666299515,
       'method': 'standardization',
       'std': 356.8056580363697},
 'z': {'mean': -0.16543275617686548,
       'method': 'standardization',
       'std': 6.410170534866934}}

else:
    # takes around 40 min
    station_features = [
        "x",
        "y",
        "z",
        "deltaTimeHottest",
        "pmt_number",
        #"WCD_signal",
        "effective_area",
        "rho_mu",  # muon density will always go latest
    ]

    massive_array = get_station_features(train_paths, station_features)
    mean_array = np.nanmean(massive_array, axis=0)
    std_array = np.nanstd(massive_array, axis=0)

    normalization_dict = {}
    for idx, feature in enumerate(station_features):
        normalization_dict[feature] = {}
        normalization_dict[feature]["method"] = "standardization"
        normalization_dict[feature]["mean"] = mean_array[idx]
        normalization_dict[feature]["std"] = std_array[idx]

    # overwrite some values that are better normalized as follows
    normalization_dict["pmt_number"] = {
        "method": "min_max_scaling",
        "min": 1,
        "max": 3,
    }
    normalization_dict["effective_area"] = {
        
        "method": "min_max_scaling",
        "min": 0,
        "max": 3 * 10.46,
    }  # Kathy Turner will have more than 3 modules in data

pprint(normalization_dict)

In [ ]:
plot_train_distributions = False

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,0], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$(x_{\text{hottest}} - x) / \text{m}$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,1], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$(y_{\text{hottest}} - y) / \text{m}$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,2], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$(z_{\text{hottest}} - z) / \text{m}$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,3], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$(t_{\text{hottest}} - t) / \text{m}$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,4], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$N_{\text{PMTs}}$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,5], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$\lg(S/\text{VEM})$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:    
    
    histogram, bin_edges = np.histogram(massive_array[:,6], bins=100, density=True)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$\text{Active Area}/m^2$"
    #ax.set_ylim(-200, 350)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

In [ ]:
if plot_train_distributions:

    histogram, bin_edges = np.histogram(massive_array[:,7], bins=10000, density=False)

    myStyle = MyStyle('1fig', markers=None)
    fig, ax = plt.subplots()

    # plot the histogram
    ax.bar(bin_edges[:-1], histogram, width=np.diff(bin_edges), edgecolor="red", alpha=0.4, color="r")
    ax.set_ylabel("norm. counts")

    #feature lim and labels
    feature_label = r"$\rho_{\mu}/\text{m}^-2$"
    ax.set_xlim(-0.2, 0)
    ax.set_xlabel(feature_label)

    plt.tight_layout()
    plt.show()

### Datasets and Loaders

In [ ]:
dir_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/JSONfiles/"
root_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/root/"

# set paths according to index
train_paths = list(train_index.iloc[:1000, 0].values)
train_paths = list(map(lambda item: dir_path + item + ".json", train_paths))

train_PyG_ds = SD433UMDatasetPyG(
    file_paths=train_paths,
    norm_dict=normalization_dict,
    root=root_path,
    sample_PMTs=True,
    sample_MD_mods=True,
    n_time_bins=60,
    neighbourhood_cut=450,
    
)

In [ ]:
def plot_graph(
    graph,
    feature_name,
    norm_dict,
    ax,
    plot_edges=True,
    node_color="cornflowerblue",
    size_amplifier=1,
    x_pos_offset=0,
    node_size_label=None,
    plot_core = False,
    linestyle="solid",
):

    def retrieve_original_value(feature, method_info):
        method = method_info["method"]
        if method == "standardization":
            return feature * method_info["std"] + method_info["mean"]
        elif method == "min_max_scaling":
            return (
                feature * (method_info["max"] - method_info["min"]) + method_info["min"]
            )
        else:
            raise ValueError(f"Unsupported method: {method}")

    # Plot edges
    if plot_edges:
        for src, dst in graph["edge_index"].t().tolist():
            ax.plot(
                [
                    graph["pos"][src, 0] + x_pos_offset,
                    graph["pos"][dst, 0] + x_pos_offset,
                ],
                [graph["pos"][src, 1], graph["pos"][dst, 1]],
                color="gray",
                linestyle=linestyle,
                alpha=0.33,
            )

    # Filter nodes based on the specified feature
    feature_index = torch.tensor(
        [i for i, feat in enumerate(graph["station_features"]) if feat == feature_name]
    )
    node_sizes = graph["x"][:, feature_index]
    # Adjust node sizes based on method
    node_sizes = retrieve_original_value(node_sizes, norm_dict[feature_name])
    # Plot nodes above edges
    ax.scatter(
        graph["pos"][:, 0] + x_pos_offset,
        graph["pos"][:, 1],
        s=size_amplifier * node_sizes,
        color=node_color,
        label=node_size_label,
        zorder=2,
    )
    
    if plot_core:
        ax.scatter(
        graph["core"][0],
        graph["core"][1], marker='x', color="green", label="MC core")
        
    delays = retrieve_original_value(graph.x[:,3], norm_dict["deltaTimeHottest"])
    # Add insets for node traces and other stuff
    for i, (x, y) in enumerate(graph["pos"]):
        ax.text(
            x.item() - 70,
            y.item() - 60,
            graph.station_list[i],
            transform=ax.transData,
            fontsize=14,
        )
        ax.text(
            x.item() - 70,
            y.item() - 120,
            f"$\Delta t_{{ \text{{hottest}} }}=$ {round(delays[i].item(), 0)} ns",
            transform=ax.transData,
            fontsize=14,
        )
        inset_ax = ax.inset_axes(
            [x.item() - 100, y.item() + 25, 200, 100], transform=ax.transData
        )
        inset_ax.plot(graph.x_traces[i, :], color="black")
        inset_ax.axis("off")

In [ ]:
def plot_graph(
    graph,
    feature_name,
    norm_dict,
    ax,
    plot_edges=True,
    node_color="cornflowerblue",
    size_amplifier=1,
    x_pos_offset=0,
    node_size_label=None,
    plot_core=False,
    edge_color="grey",
    linestyle="solid",
):

    def retrieve_original_value(feature, method_info):
        method = method_info["method"]
        if method == "standardization":
            return feature * method_info["std"] + method_info["mean"]
        elif method == "min_max_scaling":
            return (
                feature * (method_info["max"] - method_info["min"]) + method_info["min"]
            )
        else:
            raise ValueError(f"Unsupported method: {method}")

    # Plot edges
    if plot_edges:
        for src, dst in graph["edge_index"].t().tolist():
            ax.plot(
                [
                    graph["pos"][src, 0] + x_pos_offset,
                    graph["pos"][dst, 0] + x_pos_offset,
                ],
                [graph["pos"][src, 1], graph["pos"][dst, 1]],
                color=edge_color,
                linestyle=linestyle,
                alpha=0.33,
            )

    # Filter nodes based on the specified feature
    feature_index = torch.tensor(
        [i for i, feat in enumerate(graph["station_features"]) if feat == feature_name]
    )
    node_sizes = graph["x"][:, feature_index]
    # Adjust node sizes based on method
    node_sizes = retrieve_original_value(node_sizes, norm_dict[feature_name])
    # Plot nodes above edges
    ax.scatter(
        graph["pos"][:, 0] + x_pos_offset,
        graph["pos"][:, 1],
        s=size_amplifier * node_sizes,
        color=node_color,
        label=node_size_label,
        zorder=2,
    )
    
    if plot_core:
        ax.scatter(
        graph["core"][0],
        graph["core"][1], marker='x', color="green", label="MC core")
        
    delays = retrieve_original_value(graph.x[:,3], norm_dict["deltaTimeHottest"])
    # Add insets for node traces and other stuff
    for i, (x, y) in enumerate(graph["pos"]):
        ax.text(
            x.item() - 70,
            y.item() - 60,
            graph.station_list[i],
            transform=ax.transData,
            fontsize=14,
        )
        ax.text(
            x.item() - 110,
            y.item() - 120,
            f"$\Delta t_{{ \\text{{hottest}} }}=$ {round(delays[i].item(), 0)} ns",
            transform=ax.transData,
            fontsize=14,
        )
        inset_ax = ax.inset_axes(
            [x.item() - 100, y.item() + 25, 200, 100], transform=ax.transData
        )
        inset_ax.plot(graph.x_traces[i, :], color="black")
        inset_ax.axis("off")
        
def plot_simple_graph(graph, ax, kw_args_nodes, kw_args_edges, kw_args_traces, plot_edges=True, plot_traces=True, ):
    
    # plotting edges
    if plot_edges:
        
        for src, dst in graph["edge_index"].t().tolist():
            ax.plot(
                [
                    graph["pos"][src, 0],
                    graph["pos"][dst, 0],
                ],
                [graph["pos"][src, 1], graph["pos"][dst, 1]],
                **kw_args_edges
            )
            
    # plotting nodes
    for i, (x, y) in enumerate(graph["pos"]):
        
        plt.scatter(x, y, **kw_args_nodes)
        
        if plot_traces:
            inset_ax = ax.inset_axes(
            [x.item() - 100, y.item() + 25, 200, 100], transform=ax.transData
            )
            inset_ax.plot(graph.x_traces[i, :], color="black", **kw_args_traces)
            inset_ax.axis("off")

In [ ]:
train_PyG_ds = SD433UMDatasetPyG(
    file_paths=train_paths,
    norm_dict=normalization_dict,
    root=root_path,
    sample_PMTs=True,
    sample_MD_mods=True,
    n_time_bins=60,
    neighbourhood_cut=450,
    
)

graph = train_PyG_ds[10]

In [ ]:
graph

In [ ]:
graph.x

In [ ]:
myStyle = MyStyle("1fig", markers=None)
fig, ax = plt.subplots()

color="black"
kw_args_nodes = {"color":color}
kw_args_edges = {"color":color}
kw_args_traces = {}

plot_simple_graph(graph, ax=ax, kw_args_nodes=kw_args_nodes, kw_args_edges=kw_args_edges, kw_args_traces=kw_args_traces)
ax.axis("off")
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
node_masking_transform =  MaskNode(max_nodes2prune=2)
masked_graph1 = node_masking_transform(graph)

In [ ]:
myStyle = MyStyle("1fig", markers=None)
fig, ax = plt.subplots()

color="blue"
kw_args_nodes = {"color":color}
kw_args_edges = {"color":color}
kw_args_traces = {}

plot_simple_graph(masked_graph1, ax=ax, kw_args_nodes=kw_args_nodes, kw_args_edges=kw_args_edges, kw_args_traces=kw_args_traces)
ax.axis("off")
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
node_masking_transform =  MaskNode(max_nodes2prune=2)
masked_graph2 = node_masking_transform(masked_graph1)

In [ ]:
myStyle = MyStyle("1fig", markers=None)
fig, ax = plt.subplots()

color="red"
kw_args_nodes = {"color":color}
kw_args_edges = {"color":color}
kw_args_traces = {}

plot_simple_graph(masked_graph2, ax=ax, kw_args_nodes=kw_args_nodes, kw_args_edges=kw_args_edges, kw_args_traces=kw_args_traces)
ax.axis("off")
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
myStyle = MyStyle("1fig", markers=None)
fig, ax = plt.subplots()
#plot_graph(
#    graph,
#    "WCD_signal",
#    normalization_dict,
#    ax,
#    node_size_label=r"$S_{WCD}$",
#    size_amplifier=1.5,
#)
plot_graph(
    graph,
    "rho_mu",
    normalization_dict,
    ax,
    plot_edges=True,
    node_color="blue",
    size_amplifier=10,
    node_size_label=r"$\rho_{\mu}$",
    linestyle=None,
    plot_core=False,
    edge_color="blue",
)
ax.set_aspect("equal", adjustable="box")
ax.axis("off")
ax.text(x=0.01, y=0.99, s=r"$\gamma$" if graph.y == 1 else r"$p$",  ha='left', va='top', transform=ax.transAxes)
#ax.legend(loc="upper right", fontsize=14)
# plt.tight_layout()
plt.show()

In [ ]:
node_masking_transform =  MaskNode(max_nodes2prune=2)
masked_node = node_masking_transform(graph)

In [ ]:
myStyle = MyStyle("1fig", markers=None)
fig, ax = plt.subplots()
plot_graph(
    masked_node,
    "WCD_signal",
    normalization_dict,
    ax,
    node_size_label=r"$S_{WCD}$",
    size_amplifier=1,
)
plot_graph(
    masked_node,
    "rho_mu",
    normalization_dict,
    ax,
    plot_edges=False,
    node_color="red",
    size_amplifier=1.5,
    node_size_label=r"$\rho_{\mu}$",
    linestyle=None,
    plot_core=True,
)
ax.set_aspect("equal", adjustable="box")
ax.axis("off")
ax.text(x=0.01, y=0.99, s=r"$\gamma$" if graph.y == 1 else r"$p$",  ha='left', va='top', transform=ax.transAxes)
ax.legend(loc="upper right", fontsize=14)
# plt.tight_layout()
plt.show()

In [ ]:
# myStyle = MyStyle("1fig", markers=None)
# fig, ax = plt.subplots()
# station_features = [
#     "$x$",
#     "$y$",
#     "$z$",
#     r"$\Delta t_{{\text{{hottest}}}}$",
#     r"$N_{\text{PMT}}$",
#     r"$S_{\text{WCD}}$",
#     r"$A_{\mu}$",
#     r"$\rho_{\mu}$",
# ]
# # Create the table with adjusted position, size, and font size
# table_x = ax.table(
#     cellText=graph.x.numpy(),
#     colWidths=[0.3] * len(graph.station_features),
#     rowLabels=graph.station_list,
#     colLabels=station_features,
#     loc="center",
#     bbox=[0, 0, 1, 1],
#     cellLoc="center",
#     fontsize=24,
# )  # Adjust fontsize as needed
# ax.axis("off")

# plt.show()

In [ ]:
# myStyle = MyStyle("1fig", markers=None)
# fig, ax = plt.subplots()

# adj_matrix = (
#     torch_geometric.utils.to_torch_coo_tensor(graph.edge_index).to_dense().numpy()
# )
# # Create the table
# table_edges = ax.table(
#     cellText=adj_matrix,
#     colWidths=[0.1] * len(graph.station_list),
#     rowLabels=graph.station_list,
#     colLabels=graph.station_list,
#     loc="center",
#     bbox=[0, 0, 1, 1],
#     cellLoc="center",
#     fontsize=18,
# )

# # Iterate over table cells and shade non-zero cells with light grey
# for i in range(len(adj_matrix)):
#     for j in range(len(adj_matrix[i])):
#         if adj_matrix[i][j] != 0:
#             table_edges[(i + 1, j)].set_facecolor("grey")  # Light grey color

# # Turn off axis and adjust layout
# ax.axis("off")
# plt.tight_layout()
# plt.show()

In [ ]:
# myStyle = MyStyle("1fig", markers=None)
# fig, ax = plt.subplots()
# trimmed_traces = graph.x_traces[:, :25].numpy()
# table_edges = ax.table(
#     cellText=trimmed_traces,
#     colWidths=[0.1] * trimmed_traces.shape[1],
#     rowLabels=graph.station_list,
#     loc="center",
#     bbox=[0, 0, 1, 1],
#     cellLoc="center",
#     fontsize=18,
# )  # Adjust fontsize as needed
# ax.axis("off")
# plt.tight_layout()
# plt.show()

In [ ]:
dir_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/JSONfiles/"
root_path = "/sps/pauger/users/erodriguez/PhotonDiscrimination/root/"

# set paths according to index
train_paths = list(train_index.iloc[:, 0].values)
train_paths = list(map(lambda item: dir_path + item + ".json", train_paths))
val_paths = list(validation_index.iloc[:, 0].values)
val_paths = list(map(lambda item: dir_path + item + ".json", val_paths))
test_paths = list(test_index.iloc[:, 0].values)
test_paths = list(map(lambda item: dir_path + item + ".json", test_paths))

# generate Datasets
train_PyG_ds = SD433UMDatasetPyG(
    file_paths=train_paths,
    norm_dict=normalization_dict,
    root=root_path,
    sample_PMTs=True,
    sample_MD_mods=True,
)
val_PyG_ds = SD433UMDatasetPyG(
    file_paths=val_paths,
    norm_dict=normalization_dict,
    root=root_path,
    sample_PMTs=True,
    sample_MD_mods=True,
)
test_PyG_ds = SD433UMDatasetPyG(
    file_paths=test_paths,
    norm_dict=normalization_dict,
    root=root_path,
    sample_PMTs=True,
    sample_MD_mods=True,
)

In [ ]:
# generate Loaders
train_loader = DataLoader(train_PyG_ds, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(val_PyG_ds, batch_size=32, num_workers=8)
test_loader = DataLoader(test_PyG_ds, batch_size=32, num_workers=8)

In [ ]:
for batch_idx, data in enumerate(
    tqdm(train_loader, desc="Batch", position=0, leave=True)
):
    if torch.isnan(data.x).any():
        print("Batch id: ",batch_idx)
        print("Nan on Features")
        print(data.x)
        saved_problem = data.x
        nan_indices = torch.nonzero(torch.isnan(saved_problem)).flatten()
        print("Indices of NaNs:", nan_indices.tolist())
    if torch.isnan(data.x_traces).any():
        print("Batch id: ",batch_idx)
        print("Nan on Traces")
        saved_problem = data.x_traces
        nan_indices = torch.nonzero(torch.isnan(saved_problem)).flatten()
        print("Indices of NaNs:", nan_indices.tolist())
        print("Features")
        print(data.x[nan_indices[0], :])
        print("Norm. Traces after Sampling")
        print(data.x_traces[nan_indices[0],:])

In [ ]:
for batch_idx, data in enumerate(
    tqdm(val_loader, desc="Batch", position=0, leave=True)
):
    if torch.isnan(data.x).any():
        print("Batch id: ",batch_idx)
        print("Nan on Features")
        print(data.x)
        saved_problem = data.x
        nan_indices = torch.nonzero(torch.isnan(saved_problem)).flatten()
        print("Indices of NaNs:", nan_indices.tolist())
    if torch.isnan(data.x_traces).any():
        print("Batch id: ",batch_idx)
        print("Nan on Traces")
        saved_problem = data.x_traces
        nan_indices = torch.nonzero(torch.isnan(saved_problem)).flatten()
        print("Indices of NaNs:", nan_indices.tolist())
        print("Features")
        print(data.x[nan_indices[0], :])
        print("Norm. Traces after Sampling")
        print(data.x_traces[nan_indices[0],:])

In [ ]:
for batch_idx, data in enumerate(
    tqdm(test_loader, desc="Batch", position=0, leave=True)
):
    if torch.isnan(data.x).any():
        print("Batch id: ",batch_idx)
        print("Nan on Features")
        print(data.x)
        saved_problem = data.x
        nan_indices = torch.nonzero(torch.isnan(saved_problem)).flatten()
        print("Indices of NaNs:", nan_indices.tolist())
    if torch.isnan(data.x_traces).any():
        print("Batch id: ",batch_idx)
        print("Nan on Traces")
        saved_problem = data.x_traces
        nan_indices = torch.nonzero(torch.isnan(saved_problem)).flatten()
        print("Indices of NaNs:", nan_indices.tolist())
        print("Features")
        print(data.x[nan_indices[0], :])
        print("Norm. Traces after Sampling")
        print(data.x_traces[nan_indices[0],:])